In [168]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [169]:
df = pd.read_csv('sales_data.csv')
df['sale_date'] = pd.to_datetime(df['sale_date'])
df["quarter"] = df.sale_date.dt.quarter

In [170]:
def myfunc(q):

    # Filtering by quarter
    my_df = df[df.quarter == q]

    # Grouping by product_id and sum the quantities
    my_df_grouped = my_df.groupby('product_id').sum('quantity')

    # Compute the naive popularity index and scale it
    my_df_grouped['pop_index'] = my_df_grouped['quantity'] / my_df_grouped['quantity'].sum()
    my_df_grouped['pop_index_perc'] = (my_df_grouped['pop_index'] - my_df_grouped['pop_index'].min())\
                                            / (my_df_grouped['pop_index'].max() - my_df_grouped['pop_index'].min())

    # Compute the revenues index and scale it
    my_df_grouped['revenues'] = my_df_grouped['quantity'] * my_df_grouped['sales_price']
    my_df_grouped['revenues_perc'] = (my_df_grouped['revenues'] - my_df_grouped['revenues'].min())\
                                            / (my_df_grouped['revenues'].max() - my_df_grouped['revenues'].min())

    my_df_grouped.reset_index(inplace = True)
    return my_df_grouped

var = myfunc(1)

In [171]:
def ABC(X):
    top_ten = list(var[var['pop_index_perc'] > X].product_id.unique().tolist())
    my_df = var[var['product_id'].isin(top_ten)].reset_index(drop = True)
    print('Contribution to total revenues of the {} most popular products:'.format(X), my_df.revenues.sum()/var.revenues.sum())
ABC(0.1)

Contribution to total revenues of the 0.1 most popular products: 0.30147725647182294


In [173]:
def vincolo():
    print(var)


vincolo()

      product_id  quantity  sales_price_tax  regular_price_tax  sales_price  \
0         100043         3           1377.0             1530.0       1146.0   
1         100044         2           1113.0             1308.0        928.0   
2         100108         1          12990.0            14433.0      10825.0   
3         100122         4           3116.0             3512.0       2596.0   
4         100124         8          21390.0            24800.0      17826.0   
...          ...       ...              ...                ...          ...   
4782      162126         2          15980.0            17756.0      13316.0   
4783      162127         1          10990.0            12211.0       9158.0   
4784      162303         4          11460.0            13288.0       9551.0   
4785      162337         3         373320.0           466644.0     311100.0   
4786      162361         1          25990.0            28878.0      21658.0   

      regular_price  purchase_price  quarter  pop_i